# Over dit document
In dit document wordt een CRISP-DM cycles doorlopen, welke vervolgens een dataproduct (visualisaties) op zal leveren. Tot slot wordt er een conclusie getrokken en advies gegeven. Hiervoor is ter ondersteuning het Data Ethics Decision Aid model (DEDA model) voor gebruikt.

Auteur: Eefkje-Femke Jansen van der Sligte

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans, BisectingKMeans
df_original=pd.read_excel("Complete-dataset-FINAL.xlsx")

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Business Understanding
Het Isala ziekenhuis wil de zorg voor diabetespatiënten verbeteren. Het gaat hier specifiek om het behandelproces. Door onnauwkeurigheden in metingen te filteren en hierop een behandelplan aan te passen, zal er een verbetering optreden in de analyse. Visualisaties zullen helpen bij het verkrijgen van inzicht in de foutmarges van diverse meetmethoden. Op dit moment zijn er geen visualisaties die dat in kaart brengen. Om meer inzicht te bieden, willen we gaan kijken naar mogelijke relaties die er nog niet zijn. Dit zal worden gedaan door het clusteren van de Europese meetmethoden.


# Data Understanding
Welke databronnen zijn gegeven en in welk formaat: Een excel bestand genaamd: Complete-dataset-FINAL.xlsx

### note
Het excel bestand dat gebruikt gaat worden, is het resultaat van rapporten die om zijn gezet naar excel. Dit is de meest recente versie waarin de meetmethoden zijn genormaliseerd en de waarden dubbelgecheckt zijn.

Hoe groot zijn deze databronnen: 355 kB

-4685 rijen

-12 kolommen

Kolomnamen: Method name, N (no. labs), Mean, Bias, CV, Sample, Reference value, Year, Source, Type, Manufacturer (fabrikant).
- Method name : Naam van het meetmethode
- N	: Aantal deelgenomen labs
- Mean :  Gemiddelde van alle HbA1c waarde
- Bias : Absolute toe- of afname van de hoeveelheid mmol/mol ten opzichte van de werkelijke waarde
- CV : Laat zien wat de spreiding is de metingen
- Sample : Hoeveelste meting in het jaar
- Reference Value : Variabel die is gebaseerd op 95% van de gezonde populatie		
- Year : Welk jaar de meting is gedaan
- Source	: Amerikaanse of Europese data
- Type : Vers of bevroren samples

Datatypes: Object, float en int (jaar) 


In [ ]:
df_original.shape

In [ ]:
df_original.dtypes

In [ ]:
df_original.isna().sum()


In [ ]:
df_original.describe().transpose()

In [ ]:
df_original.columns

# Data Preparation

In deze fase zal het dataframe aangepast worden om een clustermodel toe te kunnen passen. 
De volgende vragen zijn als richtlijn gebruikt om de data voor te bereiden:

- Moet er nog een kolom weg?
- Zijn er aanpassingen nodig aan de data voordat we met modelling aan de slag kunnen?
- Moet de data nog genormaliseerd worden?
- Zijn er lege velden?

In [ ]:
#Scheidt de data en verwijder obect kolommen voor clustering
df_Eur=df_original.loc[df_original['Source (CAP/EurA1c)'] == 'EURA1c']
df_Eur = df_Eur.drop(columns=['N (no. labs)', 'Source (CAP/EurA1c)', 'Type (Fresh/frozen)', 'Manufacturer', 'Sample', 'Method name' , 'Total Error' , 'Bias']) # kolomm weggooien
df_Eur.columns 

In [ ]:
df_Eur.isna().sum()

Er zijn geen lege waarden te vinden in de dataset.

In [ ]:
#omzetten van object kolommen naar float kolommen voor de clustering
df_Eur["Mean"] = df_Eur["Mean"].replace("-", np.nan)
df_Eur["Mean"] = pd.to_numeric(df_Eur["Mean"])

df_Eur["CV"] = df_Eur["CV"].replace("-", np.nan)
df_Eur["CV"] = df_Eur["CV"].replace(" ", np.nan)
df_Eur["CV"] = pd.to_numeric(df_Eur["CV"])

df_Eur.info()

## Data analyse
Er is bewust de keuze gemaakt om voor de modeleer fase de data te inventariseren. Dit is alleen ter observatie en schetst een beeld van de data. Dit zal worden gedaan a.d.h.v. de 8v's. Namelijk: Volume, Velocity, Variety, Variability, Varicity, Validity, Volatility en Visualisation.


- Volume: 355 kB, 4685 rijen en 12 kolommen. Het volume zal gedurende het project niet veranderen.
- Velocity: het snel verwerken van de data is nu niet van belang. Per jaar wordt er een nieuw rapport gepubliceerd. 
- Variety: de dataset bevat meetmethodes en metingen (denk aan 'Method name'). Daarnaast zijn er nog variabelen die de meetmethodes en metingen hebben ondersteund (denk aan 'Reference value'). Tot slot zijn er nog variabelen die een meting uniek maken (denk aan 'Year' en 'Type').
- Variability: De dataset bevat medische resultaten van metingen. Deze metingen zijn statisch en de betekenis zal niet veranderen. Verder is de structuur van de data afgestemd op een dataformaat.
- Varicity: deze dataset bevat het eindresultaat van het constant valideren en checken van de kwaliteit van de data. Rijen die lege waarden bevatten, zullen niet gelijk worden verwijderd. Het gebruik van die specifieke rijen zullen bepaald worden bij de data preparation fase.  
- Validity: de gebruikte dataset is het resultaat van gepubliceerde medische rapporten die om zijn gezet naar dataframes.  
- Volatility: de dataset bevat medische resultaten van metingen. Deze metingen zijn statisch en de betekenis zal niet veranderen.
- Visualisation: voor de visualisatie worden er een pairplot en heatmap gebruikt.

In [ ]:
sns.pairplot(df_Eur)

In [ ]:
sns.heatmap(df_original.corr(),annot=True)

# Data analyse bevindingen
- In de pairplot is te zien dat de reference value een vast waarde krijgt.
- Bij de 'Mean' en 'Year' is er een range met onderbreking te zien. 
- Er zijn op het oog drie duidelijke groepen te zien bij de 'CV'-waarde en de 'Mean'.
- Er is een hoge correlatie vast te stellen bij de 'reference value' en 'Mean'. Daarnaast is er nog enigszins een correlatie vast te stellen bij de 'Bias', 'Total Error' en 'CV'-waarde. Dit is te verklaren doordat bepaalde variabelen gebruikt worden om de andere te berekenen.
- De correlatie tussen de 'Reference Value' en de 'Mean' is erg hoog, namelijk waarde 1. Note: Bij de clusteranalyse van de Amerikaanse data is er een linear verband te zien tussen de 'Mean' en 'Reference value'. Dat verband is hier niet te zien, maar verklaart de correlatie tussen deze twee variabelen wel. Namelijk dat de 'Mean' wordt gebruikt bij het berekenen van de 'Reference value'.

# Modeling


## Aantal clusters bepalen met een elbow plot

In [ ]:
df_model = df_Eur.copy()

n_clusters_to_test = range(2, 11)
inertias = []

for n in n_clusters_to_test:
    model = KMeans(n_clusters=n).fit(df_model)
    inertias.append(model.inertia_)
    
sns.lineplot(x=n_clusters_to_test, y=inertias)

## Eerste model met KMeans
De keuze om het juiste aantal clusters te kiezen, heb ik gebaseerd op het kleinste aantal clusters met de grootste knik. In dit geval is dat bij 3 clusters.


In [ ]:
test = pd.DataFrame(columns = ['centroid_x', 'centroid_y'])

model = KMeans(n_clusters=3, random_state=42).fit(df_model)
df_model['cluster'] = model.labels_
df_model['cluster'] = df_model['cluster'].astype('category')
sns.pairplot(df_model, hue='cluster')

# Analyse pair plot
De rechte banden in de onderste rij - rechts, overlappen elkaar. Hieruit kan worden geconcludeerd dat de data niet genormaliseerd hoeft te worden.

Opvallende punten bij de clusters:
- Bij cluster 0 is te zien dat de datapunten verspreid zijn over de diverse jaren. De 'CV'-waarden liggen hier aan de hoge kant. 
- Bij cluster 1 is de data vooral gefocust op de recentere jaren zoals 2018 en 2020. Hierbij zijn lagere waarden te vinden van de reference value.
- bij cluster 2 is de data vooral gefocust op de eerdere jaren zoals 2016 en 2017. Verder is te zien dat een lage 'CV'-waarde gegroupeerd wordt met hoge 'Mean'-waarden. 


# Cluster analyse

In [ ]:
cluster_0 = df_model[df_model['cluster'] == 0]
cluster_1 = df_model[df_model['cluster'] == 1]
cluster_2 = df_model[df_model['cluster'] == 2]
df_model['cluster'].value_counts()

In [ ]:
sns.boxplot(x='cluster', y='CV', data=df_model)

In [ ]:
sns.boxplot(x='cluster', y='Mean', data=df_model)

In [ ]:
sns.boxplot(x='cluster', y='Reference Value', data=df_model)

In [ ]:
sns.relplot(x='Year', y='Reference Value', hue='cluster', data=df_model)

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(10, 5), gridspec_kw=dict(width_ratios=[4, 3]))
sns.scatterplot(data=df_model, x='Year', y='Reference Value', hue="cluster", ax=axs[0])
sns.barplot(x='Year', y='Reference Value', data=df_model, hue='cluster', ax=axs[1])
f.tight_layout()

Bij deze scatterplot hebben cluster 2 en cluster 1 visueel de meeste datapunten. Wanneer hier meer wordt ingezoomd (barplot rechter visualisatie), is te zien dat cluster 0 juist alle datapunten bevat van de reference value, alleen niet zichtbaar zijn in de scatterplot door overlapping.

In [ ]:
#Add data
df_cluster_0 = pd.DataFrame(columns=['Method name', 'N (no. labs)', 'Mean', 'Bias', 'CV', 'Sample',
       'Reference Value', 'Year', 'Source (CAP/EurA1c)', 'Type (Fresh/frozen)',
       'Manufacturer', 'Total Error'])
df_cluster_0 = df_original.loc[cluster_0.index, :]
df_cluster_0['cluster'] = 0
df_cluster_0.describe().transpose()

In [ ]:
df_cluster_1 = pd.DataFrame(columns=['Method name', 'N (no. labs)', 'Mean', 'Bias', 'CV', 'Sample',
       'Reference Value', 'Year', 'Source (CAP/EurA1c)', 'Type (Fresh/frozen)',
       'Manufacturer', 'Total Error'])
df_cluster_1 = df_original.loc[cluster_1.index, :]
df_cluster_1['cluster'] = 1
df_cluster_1.describe().transpose()

In [ ]:
df_cluster_2 = pd.DataFrame(columns=['Method name', 'N (no. labs)', 'Mean', 'Bias', 'CV', 'Sample',
       'Reference Value', 'Year', 'Source (CAP/EurA1c)', 'Type (Fresh/frozen)',
       'Manufacturer', 'Total Error'])
df_cluster_2 = df_original.loc[cluster_2.index, :]
df_cluster_2['cluster'] = 2
df_cluster_2.describe().transpose()

In [ ]:
clustered_df= pd.concat([df_cluster_0, df_cluster_1, df_cluster_2], ignore_index=True)

In [ ]:
clustered_df.groupby('cluster')['Manufacturer'].value_counts()

In [ ]:
clustered_df.groupby('cluster')['Type (Fresh/frozen)'].value_counts()

# Evaluation

Tijdens het uitwerken van de clusteranalyse is bij het valideren met de product owner naar voren gekomen, dat de Amerikaanse en Europese data niet met elkaar te vergelijken zijn. Als gevolg hiervan heb ik nieuwe clusteranalyses uitgevoerd waarbij deze data gescheiden van elkaar zijn. 

# Analyse uitwerking van clusters

Bij het clusteren van de Europese data is in de pairplot een goede scheiding te zien tussen de clusters 1 & 2 en cluster 0. In de plot is te zien dat de hoogste CV-waarden zijn verdeeld in cluster 0. Verder is de range van de 'Mean'-waarden door de jaren heen uit elkaar gelopen om vervolgens in 2020 samengevoegd te zijn.

Verdere bevindingen:
- Cluster 0 bevat datapunten vanuit alle jaren. Daarnaast zijn in dit cluster de hogere 'CV'-waarden ingedeeld.
- In cluster 2 zijn de hogere 'Mean'-waarden van eerdere jaren te vinden.
- In de clusterverdeling is te zien dat er een onderscheid tussen de eerdere metingen en meer recentere metingen lijkt te zijn. Waarbij cluster 2 oudere metingen bevat en cluster 1 de meest actuele metingen.
- De manufacturers zijn gelijkmatig verdeeld tussen de clusters.
- De verhouding van het type tot de clustering lijkt ook in evenwicht te zijn. De grotere clusters bevatten meer 'Frozen' samples dan het kleinere cluster.


# Conclusie

Om meer inzicht te bieden, willen we gaan kijken naar mogelijke relaties die er nog niet zijn. Dit is gedaan door middel van het clusteren van de Europese meetmethoden. In de clusteranalyse zijn geen nieuwe relaties gelegd.  


# Advies

In de bevindingen die zijn gedaan, kunnen geen concrete relaties gelegd worden. Hiermee is het advies dan ook om dit model, de gemaakte bevindingen en de conclusie als aanleiding te gebruiken voor andere onderzoeken. 


Een vervolgonderzoek zou de invloed van 'Year' op de clustering zijn. 


#### Invloed van 'Year' op de clustering
Bij de clusterindeling zijn er 3 clusters gevormd. Eén met eerdere metingen, één met de meest actuele metingen en het laatste cluster bevat metingen van alle jaren, waarbij het lijkt dat de 'CV'-waarde hier aan de hoge kant ligt. Als advies geef ik mee om in een vervolg onderzoek/clusteranalyse te gaan kijken naar de invloed van het 'Year' op de clusterindeling. Hier zou een andere clusterindeling kunnen komen waarbij de focus meer ligt op andere variabelen en daardoor mogelijke andere relaties gelegd kunnen worden.



#### Ethische aspecten 
Om verantwoordelijk om te gaan met data is het belangrijk om een ethische invalshoek bij het proces te betrekken.  

Integriteit

Daarnaast is 'integriteit' een belangrijke waarde bij het beheren van data. We hebben naar deze waarde zoveel mogelijk gestreeft. De HbA1c meetmethoden zijn, door de jaren heen, veel veranderd. Zo zijn benaming van meetmethoden veranderd, sommige meetmethoden hielden op en er zijn later nieuwe meetmethoden bij gekomen. Hierbij valt desondanks niet met zekerheid te zeggen dat de meetmethoden geheel 100% nauwkeurig genormaliseerd zijn. Bij het normaliseren zat veel onderzoek achter en door veel desk research zijn is er een meer complete dataset te ontwikkelen. 

Privacy

Ten slotte is de waarde 'privacy' hier geheel mee rekening gehouden. Alleen openbare gepubliceerde rapporten zijn gebruikt om deze voorspelling te realiseren. Hier is verder geen data te herleiden naar patiënteninformatie. Verder zijn de fabrikantnamen wel publiekelijk. 

 